DataFlair

# Human Activity Recognition

Activities:
Sitting,
Standing,
Walking,
Upstairs,
Downstairs

Sensors:
Accelerometer,
Gyroscope

In [7]:
import numpy as np

In [8]:
Signal_Types = [   #These are all the distinct types of signals in the dataset
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]


In [9]:
# Creating functions to load X_train, X_test, y_train and y_test from the dataset

def get_X(X_signals_paths):
    X_signals = []

    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()

    return np.transpose(np.array(X_signals), (1, 2, 0))


def get_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()

    
    return y_ - 1 # Subtracting 1 for zero based indexing

In [10]:
# Defining paths and using them to load the data into X_train, X_test, y_train and y_test from the dataset.

Data_path = "C:\\Python programming\\DataFlair\\HAR\\human+activity+recognition+using+smartphones\\UCI HAR Dataset"

X_train_signals_paths = [
    Data_path + "\\train" + "\\Inertial Signals\\" + signal + "train.txt" for signal in Signal_Types
]

X_test_signals_paths = [
    Data_path + "\\test" + "\\Inertial Signals\\" + signal + "test.txt" for signal in Signal_Types
]

X_train = get_X(X_train_signals_paths)
X_test = get_X(X_test_signals_paths)


y_train_path = Data_path + "\\train\\" + "y_train.txt"
y_test_path = Data_path + "\\test\\" + "y_test.txt"

y_train = get_y(y_train_path)
y_test = get_y(y_test_path)

In [11]:
# One-hot encoding y_train and y_test

from keras.utils import to_categorical

y_train_one_hot = to_categorical(y_train, num_classes=6)
y_test_one_hot = to_categorical(y_test, num_classes=6)

In [12]:
# Building and traiing the LSTM model.

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam

model = Sequential()
model.add(LSTM(units=64, input_shape=(128, 9), return_sequences=True))
model.add(Dropout(0.7))
model.add(LSTM(units=64))
model.add(Dropout(0.7))
model.add(Dense(units=6, activation='softmax'))

# Compiling the model
optimizer = Adam(learning_rate=0.0025)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("DataFlair")

# Printing the model summary
model.summary()

# Training the model
model.fit(X_train, y_train_one_hot, epochs=20, batch_size=32, validation_data=(X_test, y_test_one_hot))

# Evaluating the model on the test set
loss, accuracy = model.evaluate(X_test, y_test_one_hot)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')


DataFlair
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128, 64)           18944     
                                                                 
 dropout_2 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 52358 (204.52 KB)
Trainable params: 52358 (204.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 